# Final Project ML
## Group 12
### Name: Manthan Patel

In [1]:
#Importing the modules
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data pre-processing

In [4]:
# loading the dataset to a pandas DataFrame
dataset = pd.read_csv("D:\Guelph_study\ML\Project\data.csv", low_memory = False)

In [5]:
data=dataset.iloc[:,0:5]

In [6]:
data

,title,text,subject,date,label
0,Bye Bye Cowboys! Crowd Boos As Owner Jerry Jon...,The Dallas Cowboys tried to have it both ways ...,Government News,25-Sep-17,1
1,U.S. lawmakers seek missing information in rev...,"LONDON, (Reuters) - The chairman of a congress...",politicsNews,8-Aug-17,0
2,SELF-ADMITTED SEXUAL PREDATOR Who Supported Wi...,The hypocrisy of these liberal entertainers ...,politics,20-Nov-16,1
3,Trump travel curbs pose revenue challenges for...,NEW YORK/SAN FRANCISCO (Reuters) - President D...,politicsNews,1-Feb-17,0
4,MARINE ARRESTED FOR Complaining About Governme...,This is some pretty surreal stuff In the four ...,Government News,27-Nov-15,1
...,...,...,...,...,...
44943,Stephen Colbert Trolls Trump By Announcing Ru...,Is he really going to do it this time? With St...,News,24-Jun-17,1
44944,LEFTIST ALAN COLMES THINKS WE SHOULD STOP “USI...,Remember when the left would have been ashamed...,left-news,5-May-15,1
44945,The American Energy Success Story Obama Won’t ...,Obama has waisted billions on green energy but...,Government News,29-Nov-15,1
44946,Bill O’Reilly SCREWS Fox News As SHOCKING Amo...,One of Fox News biggest and most successful n...,News,4-Apr-17,1


In [7]:
# counting the number of missing values in the dataset
data.isnull().sum()

title      44
text       49
subject    70
date       70
label      70
dtype: int64

In [8]:
# replacing the null values with empty string
data = data.fillna('')

In [9]:
data['content'] = data['title']+' '+ data['subject']
print(data['content'])

0        Bye Bye Cowboys! Crowd Boos As Owner Jerry Jon...
1        U.S. lawmakers seek missing information in rev...
2        SELF-ADMITTED SEXUAL PREDATOR Who Supported Wi...
3        Trump travel curbs pose revenue challenges for...
4        MARINE ARRESTED FOR Complaining About Governme...
                               ...                        
44943     Stephen Colbert Trolls Trump By Announcing Ru...
44944    LEFTIST ALAN COLMES THINKS WE SHOULD STOP “USI...
44945    The American Energy Success Story Obama Won’t ...
44946     Bill O’Reilly SCREWS Fox News As SHOCKING Amo...
44947     Major City SLAPS Greedy CEOs Like Trump With ...
Name: content, Length: 44948, dtype: object


In [10]:
# separating the data & label
X = data.drop(columns='label', axis=1)
Y = data['label']

In [11]:
print(X)
print(Y)

                                                   title  \
0      Bye Bye Cowboys! Crowd Boos As Owner Jerry Jon...   
1      U.S. lawmakers seek missing information in rev...   
2      SELF-ADMITTED SEXUAL PREDATOR Who Supported Wi...   
3      Trump travel curbs pose revenue challenges for...   
4      MARINE ARRESTED FOR Complaining About Governme...   
...                                                  ...   
44943   Stephen Colbert Trolls Trump By Announcing Ru...   
44944  LEFTIST ALAN COLMES THINKS WE SHOULD STOP “USI...   
44945  The American Energy Success Story Obama Won’t ...   
44946   Bill O’Reilly SCREWS Fox News As SHOCKING Amo...   
44947   Major City SLAPS Greedy CEOs Like Trump With ...   

                                                    text          subject  \
0      The Dallas Cowboys tried to have it both ways ...  Government News   
1      LONDON, (Reuters) - The chairman of a congress...     politicsNews   
2      The hypocrisy of these liberal  entertain

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [12]:
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [23]:
data['content'] = data['content'].apply(stemming)

In [24]:
print(data['content'])

0        bye bye cowboy crowd boo owner jerri jone knee...
1        u lawmak seek miss inform review monsanto weed...
2        self admit sexual predat support wife enabl se...
3        trump travel curb pose revenu challeng u colle...
4        marin arrest complain govern facebook su gover...
                               ...                        
44943    stephen colbert troll trump announc run presid...
44944    leftist alan colm think stop use nation anthem...
44945    american energi success stori obama tell world...
44946    bill reilli screw fox news shock amount advert...
44947    major citi slap greedi ceo like trump tax pena...
Name: content, Length: 44948, dtype: object


In [25]:
data

,title,text,subject,date,label,content
0,Bye Bye Cowboys! Crowd Boos As Owner Jerry Jon...,The Dallas Cowboys tried to have it both ways ...,Government News,25-Sep-17,1,bye bye cowboy crowd boo owner jerri jone knee...
1,U.S. lawmakers seek missing information in rev...,"LONDON, (Reuters) - The chairman of a congress...",politicsNews,8-Aug-17,0,u lawmak seek miss inform review monsanto weed...
2,SELF-ADMITTED SEXUAL PREDATOR Who Supported Wi...,The hypocrisy of these liberal entertainers ...,politics,20-Nov-16,1,self admit sexual predat support wife enabl se...
3,Trump travel curbs pose revenue challenges for...,NEW YORK/SAN FRANCISCO (Reuters) - President D...,politicsNews,1-Feb-17,0,trump travel curb pose revenu challeng u colle...
4,MARINE ARRESTED FOR Complaining About Governme...,This is some pretty surreal stuff In the four ...,Government News,27-Nov-15,1,marin arrest complain govern facebook su gover...
...,...,...,...,...,...,...
44943,Stephen Colbert Trolls Trump By Announcing Ru...,Is he really going to do it this time? With St...,News,24-Jun-17,1,stephen colbert troll trump announc run presid...
44944,LEFTIST ALAN COLMES THINKS WE SHOULD STOP “USI...,Remember when the left would have been ashamed...,left-news,5-May-15,1,leftist alan colm think stop use nation anthem...
44945,The American Energy Success Story Obama Won’t ...,Obama has waisted billions on green energy but...,Government News,29-Nov-15,1,american energi success stori obama tell world...
44946,Bill O’Reilly SCREWS Fox News As SHOCKING Amo...,One of Fox News biggest and most successful n...,News,4-Apr-17,1,bill reilli screw fox news shock amount advert...


In [26]:
#separating the data and label
X = data['content']
Y = data['label']

In [27]:
X

0        bye bye cowboy crowd boo owner jerri jone knee...
1        u lawmak seek miss inform review monsanto weed...
2        self admit sexual predat support wife enabl se...
3        trump travel curb pose revenu challeng u colle...
4        marin arrest complain govern facebook su gover...
                               ...                        
44943    stephen colbert troll trump announc run presid...
44944    leftist alan colm think stop use nation anthem...
44945    american energi success stori obama tell world...
44946    bill reilli screw fox news shock amount advert...
44947    major citi slap greedi ceo like trump tax pena...
Name: content, Length: 44948, dtype: object

In [28]:
pd.DataFrame(X,Y)

,content
label,
1,NaN
0,NaN
1,NaN
0,NaN
1,NaN
...,...
1,NaN
1,NaN
1,NaN


In [29]:
print(X)

0        bye bye cowboy crowd boo owner jerri jone knee...
1        u lawmak seek miss inform review monsanto weed...
2        self admit sexual predat support wife enabl se...
3        trump travel curb pose revenu challeng u colle...
4        marin arrest complain govern facebook su gover...
                               ...                        
44943    stephen colbert troll trump announc run presid...
44944    leftist alan colm think stop use nation anthem...
44945    american energi success stori obama tell world...
44946    bill reilli screw fox news shock amount advert...
44947    major citi slap greedi ceo like trump tax pena...
Name: content, Length: 44948, dtype: object


In [30]:
print(Y)

0        1
1        0
2        1
3        0
4        1
        ..
44943    1
44944    1
44945    1
44946    1
44947    1
Name: label, Length: 44948, dtype: object


In [31]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [32]:
print(X)

  (0, 12492)	0.08718981191656512
  (0, 11466)	0.2043100765925704
  (0, 8281)	0.2190361310227432
  (0, 7807)	0.0657237297471734
  (0, 6343)	0.2687120278129719
  (0, 6140)	0.24612871129910985
  (0, 6081)	0.2824355157943186
  (0, 4881)	0.131387000923953
  (0, 2689)	0.21507576060196137
  (0, 2590)	0.3066084567791501
  (0, 1642)	0.5648710315886372
  (0, 1386)	0.30350317118053394
  (0, 1318)	0.25417161968988006
  (0, 453)	0.22908084790539354
  (1, 12716)	0.5062003582691564
  (1, 10215)	0.26756844093997767
  (1, 9661)	0.3133923189907986
  (1, 8777)	0.11364790695059057
  (1, 7513)	0.5255364802501161
  (1, 7413)	0.31686212125374835
  (1, 6516)	0.2630911577273614
  (1, 5813)	0.3394563022113711
  (2, 12915)	0.19504280839527902
  (2, 12812)	0.1828003135976377
  (2, 12780)	0.11843370838773501
  :	:
  (44945, 361)	0.30241540195752326
  (44946, 12004)	0.21240717147956095
  (44946, 10399)	0.26504362962167083
  (44946, 10161)	0.34429292391490346
  (44946, 9465)	0.31977056850397567
  (44946, 7807)	0.179

# Splitting the dataset to training & test data

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=3)

Training the Model: Logistic Regression

In [34]:
model = LogisticRegression()

In [35]:
model.fit(X_train, Y_train)

LogisticRegression()

Accuracy Score

In [36]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [37]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9982836983123033


In [38]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [39]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9982944011865035


Making a Predictive System

In [40]:
X_pred = X_test[782]

prediction = model.predict(X_pred)
print(prediction)

if(prediction[0]=='0'):
    print('The news is Real')
else:
    print('The news is Fake')

['1']
The news is Fake
